In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import sympy as sp
import math
# !pip3 install git+https://github.com/Fangyh09/pytorch-receptive-field.git
from torch_receptive_field import receptive_field
from torchinfo import summary

import models as cifar_models

In [2]:
model = cifar_models.NetBN()

counter = 0
# Loop through each layer of the model
c_in, h_in, w_in = 3, 32, 32
for name, layer in model.named_children():
    # print(f"- Layer details: {name} | k={layer.kernel_size} | s={layer.stride} | p={layer.padding}")
    print(f"- Layer: {name}")
    if isinstance(layer, nn.Conv2d):
        out_channels = layer.out_channels
        k, s, p = layer.kernel_size[0], layer.stride[0], layer.padding[0]
        c_o, h_o, w_o = out_channels, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}")
        c_in, h_in, w_in = c_o, h_o, w_o
    
    elif isinstance(layer, nn.MaxPool2d):
        k, s, p = layer.kernel_size, layer.stride, layer.padding
        c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
        c_in, h_in, w_in = c_o, h_o, w_o

    elif isinstance(layer, nn.AvgPool2d):
        k, s, p = layer.kernel_size, layer.stride, layer.padding
        c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
        c_in, h_in, w_in = c_o, h_o, w_o

    elif isinstance(layer, nn.Sequential):
        for sub_name, sub_layer in layer.named_children():
            # print(f"- SubLayer details: {sub_name} | k={sub_layer.kernel_size} | s={sub_layer.stride} | p={sub_layer.padding}")
            if isinstance(sub_layer, nn.ReLU):
                continue
            print(f"- SubLayer: {sub_layer}")
            if isinstance(sub_layer, nn.Conv2d):
                out_channels = sub_layer.out_channels
                k, s, p = sub_layer.kernel_size[0], sub_layer.stride[0], sub_layer.padding[0]
                c_o, h_o, w_o = out_channels, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
                print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}")
                c_in, h_in, w_in = c_o, h_o, w_o
            
            elif isinstance(sub_layer, nn.MaxPool2d):
                k, s, p = sub_layer.kernel_size, sub_layer.stride, sub_layer.padding
                c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
                print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
                c_in, h_in, w_in = c_o, h_o, w_o

            elif isinstance(sub_layer, nn.AvgPool2d):
                k, s, p = sub_layer.kernel_size, sub_layer.stride, sub_layer.padding
                c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
                print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
                c_in, h_in, w_in = c_o, h_o, w_o
    else:
        print('layer not implemented')
        break


- Layer: convBlock1
- SubLayer: Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
-------------------------------- no. of channels: 8 | feature map dim: 32x32
- SubLayer: BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
- Layer: convBlock2
- SubLayer: Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
-------------------------------- no. of channels: 16 | feature map dim: 32x32
- SubLayer: BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
- Layer: convBlock3
- SubLayer: Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
-------------------------------- no. of channels: 8 | feature map dim: 32x32
- SubLayer: BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
- Layer: pool1
-------------------------------- no. of channels: 8 | feature map dim: 16x16

- Layer: convBlock4
- SubLayer: Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
------------------------------

In [3]:
# receptive_field(model, input_size=(channels, H, W))
receptive_field(model, input_size=(3, 32, 32))

------------------------------------------------------------------------------
        Layer (type)    map size      start       jump receptive_field 
        0               [32, 32]        0.5        1.0             1.0 
        1               [32, 32]        0.5        1.0             3.0 
        2               [32, 32]        0.5        1.0             3.0 
        3               [32, 32]        0.5        1.0             3.0 
        4               [32, 32]        0.5        1.0             5.0 
        5               [32, 32]        0.5        1.0             5.0 
        6               [32, 32]        0.5        1.0             5.0 
        7               [32, 32]        0.5        1.0             5.0 
        8               [32, 32]        0.5        1.0             5.0 
        9               [32, 32]        0.5        1.0             5.0 
        10              [16, 16]        1.0        2.0             6.0 
        11              [16, 16]        1.0        2.0   

OrderedDict([('0',
              OrderedDict([('j', 1.0),
                           ('r', 1.0),
                           ('start', 0.5),
                           ('conv_stage', True),
                           ('output_shape', [-1, 3, 32, 32])])),
             ('1',
              OrderedDict([('j', 1.0),
                           ('r', 3.0),
                           ('start', 0.5),
                           ('input_shape', [-1, 3, 32, 32]),
                           ('output_shape', [-1, 8, 32, 32])])),
             ('2',
              OrderedDict([('j', 1.0),
                           ('r', 3.0),
                           ('start', 0.5),
                           ('input_shape', [-1, 8, 32, 32]),
                           ('output_shape', [-1, 8, 32, 32])])),
             ('3',
              OrderedDict([('j', 1.0),
                           ('r', 3.0),
                           ('start', 0.5),
                           ('input_shape', [-1, 8, 32, 32]),
             

In [4]:
model.summary(input_size=(1, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
NetBN                                    [1, 10]                   --
├─Sequential: 1-1                        [1, 8, 32, 32]            --
│    └─Conv2d: 2-1                       [1, 8, 32, 32]            224
│    └─BatchNorm2d: 2-2                  [1, 8, 32, 32]            16
│    └─ReLU: 2-3                         [1, 8, 32, 32]            --
├─Sequential: 1-2                        [1, 16, 32, 32]           --
│    └─Conv2d: 2-4                       [1, 16, 32, 32]           1,168
│    └─BatchNorm2d: 2-5                  [1, 16, 32, 32]           32
│    └─ReLU: 2-6                         [1, 16, 32, 32]           --
├─Sequential: 1-3                        [1, 8, 32, 32]            --
│    └─Conv2d: 2-7                       [1, 8, 32, 32]            136
│    └─BatchNorm2d: 2-8                  [1, 8, 32, 32]            16
│    └─ReLU: 2-9                         [1, 8, 32, 32]            --
├─MaxPool2